<a href="https://colab.research.google.com/github/iMan0405/Portfolio_DT_vs_AI/blob/main/Sentiment_Analysis_for_Social_Media.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentiment Analysis modeli (Ijtimoiy tarmoq uchun)

### ushbu modelda twitter dataseti dan foydalanamiz va izohlarda qoldirilgan hissiyotlarni tahlil qilamiz:

Dataset haqida ma'lumot:
**textID** - matnning har bir qismi uchun noyob identifikator;
**text** - izoh matni;
**sentiment** - izohga berilgan hissiyot.